In [50]:
import nltk
from collections import Counter
from nltk import ngrams
from Levenshtein import distance
import string
from nltk import pos_tag
from gensim.models import KeyedVectors
from collections import defaultdict

# Download necessary NLTK data
nltk.download('words')
nltk.download('brown')

from nltk.corpus import words, brown

[nltk_data] Downloading package words to /home/nathalie/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /home/nathalie/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [51]:
vocabulary = set(words.words())
word_frequencies = Counter(brown.words())

In [52]:
def preprocess_text(text):
    """Remove punctuation and lowercase text."""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator).lower()

In [53]:
tokens = [word.lower() for word in brown.words()]

In [54]:
model = Counter(ngrams(tokens, 3))

In [ ]:
word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus

model = Counter(ngrams(tokens, 3))

input = 'my friend anne is dast runner'
input_preprocessed = preprocess_text(input)
words = input_preprocessed.split()

corrected_words = []
prev_word = None

for i, word in enumerate(words):
    suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
    suggestions = [suggestion[0] for suggestion in suggestions[0:5]]

    # Initialize variables
    best_suggestion = suggestions[0]
    best_combined_prob = 0

    # Get the next word (if it exists)
    next_word = words[i + 1] if i + 1 < len(words) else None

    for suggestion in suggestions:
        # Calculate bigram probability with the previous word
        prob_prev = 0
        if prev_word:
            bigram_count = model.get((prev_word, suggestion), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == prev_word)
            prob_prev = bigram_count / unigram_count if unigram_count > 0 else 0

        # Calculate bigram probability with the next word
        prob_next = 0
        if next_word:
            bigram_count = model.get((suggestion, next_word), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == suggestion)
            prob_next = bigram_count / unigram_count if unigram_count > 0 else 0

        # Combine probabilities (you can adjust weights as needed)
        combined_prob = (prob_prev + prob_next) / 2

        # Update best suggestion based on combined probability
        if combined_prob > best_combined_prob:
            best_combined_prob = combined_prob
            best_suggestion = suggestion

    corrected_words.append(best_suggestion)
    prev_word = best_suggestion  # Update previous word

corrected_text = ' '.join(corrected_words).capitalize() + '.'

doc = nlp(corrected_text)
for ent in doc.ents:
    corrected_text = corrected_text.replace(ent.text, ent.text.capitalize())

print(corrected_text)

My friend Anne is last runner.


In [68]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("My friend anne is a fast runner")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


kyle 10 14 PERSON
